In [1]:
class knn:
    dados = []
    labels = []
    classes = []
    
  
    def __init__(self, **args):
        self.funcao_calculo = self.calculo_de
        self.k = 3
        
        if 'k' in args:
            self.k = args['k']
            
        if 'func' in args:
            n_funcao = args['func']            
            if n_funcao == 'x2':
                self.funcao_calculo = self.calculo_x2
            elif n_funcao == 'de':
                self.funcao_calculo = self.calculo_de
            else:
                raise Exception('Funcao invalida')
        
    def calculo_de(self, v1, v2):
        if(len(v1) != len(v2)):
            raise Exception('O Tamanho dos vetores devem ser iguais')
        soma = 0
        for i in range(len(v1)):
            soma += pow(float(v2[i]) - float(v1[i]), 2)
        return pow(soma, 0.5)
    
    def calculo_x2(self, v1, v2):
        if(len(v1) != len(v2)):
            raise Exception('O Tamanho dos vetores devem ser iguais')
       
        soma = 0 
        for i in range(len(v1)):
            soma += (pow(float(v2[i]) - float(v1[i]), 2) / (float(v2[i]) + float(v1[i])))
        return soma * 0.5
    
    def calcular_distancias(self, matriz, classes, vetor):    
        distancias = []
        for i in range(len(matriz)):
            distancias.append([self.funcao_calculo(matriz[i], vetor),classes[i]])
        return distancias

    def achar_classes(self, labels):
        classes = set()
        for label in labels:
            classes.add(label)
        return classes

    def predict(self, matriz):
        labels = []
        for linha in matriz:        
            distancias_linha = self.calcular_distancias(self.dados, self.labels , linha)            
            labels.append(self.contar_classes(distancias_linha))            
        return labels
    
    def montar_mapa_classes(self, kdistancias):
        mapa = {}
        for linha in kdistancias:
            if linha[1] in mapa:
                mapa[linha[1]] += 1
            else:
                mapa.update({linha[1]:1})   
        return mapa
    
    def fit(self,X, y):        
        self.dados = X
        self.labels = y       
        return self
    
    def score(self, X, y=None):
        predito = self.predict(X)
        acertos = 0
        for y_teste, y_predito in zip( y, predito):
            if y_teste == y_predito:
                acertos += 1
        return acertos / len(y)
        
    
    def get_params(self, deep = False):
        return {'k':self.k}
       
    
    def contar_classes(self, distancias):        
        distancias.sort()
        mapa = self.montar_mapa_classes(distancias[:self.k])           
        s = [(k, mapa[k]) for k in sorted(mapa, key = mapa.get, reverse=True)]        
        return s[0][0]

In [28]:
import csv
from random import seed
from random import randrange


dados=[]
target=[]

with open('iris.data', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
#     cabecalho = next(csv_reader)
    for line in csv_reader:
        if len(line) != 0:        
            dados.append(line[0:4])
            target.append(line[4])
            
def carregar_csv(nome,col_features,col_classes):
    dados=[]
    target=[]

    with open(nome, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
    #     cabecalho = next(csv_reader)
        for line in csv_reader:
            if len(line) != 0:        
                dados.append(line[col_features])
                target.append(line[col_classes])
    return dados,target
    

d,t = carregar_csv('wine.data',slice(1,14),0)


 
def train_test_split(dataset, target, per_teste=0.60, semente = 40):
    seed(semente)      
    X_train = list(dataset)
    x_test = list()
    y_train = list(target)
    y_test = list()
    test_size = per_teste * len(dataset)   
    
    while len(x_test) < test_size:
        index = randrange(len(X_train))
        x_test .append(X_train.pop(index))
        y_test.append(y_train.pop(index))    
    
    return X_train, x_test, y_train, y_test


X_train, x_test, y_train, y_test = train_test_split(d, t, per_teste = 0.40)



In [32]:
modelo = knn(k = 15, func='x2')
modelo.fit(X_train, y_train)

In [33]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

predict = modelo.predict(x_test)

print(metrics.classification_report(y_test, predict))
print(metrics.confusion_matrix(y_test, predict))
print(modelo.score(x_test,y_test))

score = cross_val_score(modelo, dados, target, cv=10)
print(score.mean())

              precision    recall  f1-score   support

           1       0.75      1.00      0.86        24
           2       0.96      0.71      0.81        31
           3       0.71      0.71      0.71        17

   micro avg       0.81      0.81      0.81        72
   macro avg       0.80      0.81      0.79        72
weighted avg       0.83      0.81      0.80        72

[[24  0  0]
 [ 4 22  5]
 [ 4  1 12]]
0.8055555555555556
0.9266666666666667
